# Геолокация сообщений

#### Импорт библиотек

In [1]:
import warnings

warnings.simplefilter("ignore")

import os
import sys

folder = os.getcwd().split("\\examples")[0]
sys.path.append(folder)

import sloyka
from sloyka import Geocoder

INFO: Pandarallel will run on -1 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


[nltk_data] Downloading package stopwords to /home/gk/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


#### Загрузка данных

In [2]:
import pandas as pd

df = pd.read_csv('/home/gk/vscode/SOIKA_2/sloyka/sloyka/sample_data/sample_data.csv', index_col=0)

In [3]:
df.head()

,date,text,link
0,2024-02-29 08:00:02,А какой ваш любимый фрукт?Я бы персиков поела....,NaN
1,2024-02-29 07:00:04,27.02. 2024г.На территории Ленсоветовский откр...,NaN
2,2024-02-29 06:00:03,"Требуется продавец продовольственных товаров,п...",NaN
3,2024-02-28 21:05:00,#длятехктонеспит Нашла фильм 29 февраля)Корея....,NaN
6,2024-02-28 16:30:00,"🙋🏼Дорогие соседи, хотим поделиться с вами конт...",https://vk.com/remontholodilnikovvspb


#### Запуск геолокатора со встроенным геокодером

In [ ]:
result = Geocoder().run(df[:20], text_column='text')

In [21]:
pd.set_option('display.max_colwidth', None)
pd.reset_option('display.width', None)
pd.set_option('display.width', 2000)


In [4]:
mask = (result.Score.isna())&(~result.Street.isna())
result[mask]
# result.to_csv('C:\Users\USER\SOIKA\сохры.csv', index=False)

,message_id,Дата и время,Текст комментария,Street,initial_street,only_full_street_name,Numbers,Score,location_options,Location,geometry,level
9,9,2023.01.13 20:58,а метро Адмиралтейская почти постоянно стоит 2...,адмиралтейская,Адмиралтейская,Адмиралтейская набережная,,NaN,[Адмиралтейская набережная Санкт-Петербург Ро...,"Адмиралтейская набережная, Адмиралтейский окру...",POINT (30.30984 59.93990),street
9,9,2023.01.13 20:58,а метро Адмиралтейская почти постоянно стоит 2...,адмиралтейская,Адмиралтейская,Адмиралтейская улица,,NaN,[Адмиралтейская набережная Санкт-Петербург Ро...,"Адмиралтейская набережная, Адмиралтейский окру...",POINT (30.30984 59.93990),street
10,10,2023.01.13 20:41,Б. Московская-идти невозможно-каток из подтаяв...,адмиралтейская,Адмиралтейская,Адмиралтейская набережная,,NaN,[Адмиралтейская набережная Санкт-Петербург Ро...,"Адмиралтейская улица, Чухонка, Колпино, Санкт-...",POINT (30.59423 59.74329),street
10,10,2023.01.13 20:41,Б. Московская-идти невозможно-каток из подтаяв...,адмиралтейская,Адмиралтейская,Адмиралтейская улица,,NaN,[Адмиралтейская набережная Санкт-Петербург Ро...,"Адмиралтейская улица, Чухонка, Колпино, Санкт-...",POINT (30.59423 59.74329),street
11,11,2023.01.15 15:37,Это просто кошмар какой-то. Дворы вообще не уб...,социалистическая,Социалистическая,Социалистическая улица,,NaN,[Социалистическая улица Санкт-Петербург Россия],"Социалистическая улица, Ушково, Санкт-Петербур...",POINT (29.64438 60.24111),street


In [12]:
from shapely.geometry import Point

result['geometry'] = result.Location.progress_apply(lambda x: Point(x.longitude, x.latitude))

100%|██████████| 22/22 [00:00<00:00, 6228.46it/s]


#### Визаулизация полученных сообщений

In [13]:
result.head(3)

,message_id,Дата и время,Текст комментария,Street,Score,Numbers,initial_street,Toponims,full_street_name,location_options,addr_to_geocode,only_full_street_name,Location,geometry
0,0,2023.01.26 16:32,Здравствуйте! В Санкт-Петербурге нет Генеральн...,мойка,0.997,11,Мойка,улица,набережная реки Мойки 11 Санкт-Петербург Россия,[набережная реки Мойки 11 Санкт-Петербург Россия],набережная реки Мойки 11 Санкт-Петербург Россия,набережная реки Мойки,"(Генеральное консульство Нидерландов, 11, набе...",POINT (30.3245412 59.9425431)
1,1,2023.01.26 11:55,[club143265175|Центральный район Санкт-Петербу...,рубинштейна,0.900,2,Рубинштейна,None,улица Рубинштейна 2 Санкт-Петербург Россия,[улица Рубинштейна 2 Санкт-Петербург Россия],улица Рубинштейна 2 Санкт-Петербург Россия,улица Рубинштейна,"(2/45, улица Рубинштейна, Владимирский округ, ...",POINT (30.34623809680399 59.9324291)
2,2,2023.01.28 12:39,"1) Фурштатская, 19 Отслоение штукатурного слоя...",фурштатская,0.784,19,Фурштатская,None,Фурштатская улица 19 Санкт-Петербург Россия,[Фурштатская улица 19 Санкт-Петербург Россия],Фурштатская улица 19 Санкт-Петербург Россия,Фурштатская улица,"(Онлайн Трейд, 19, Фурштатская улица, Литейный...",POINT (30.3559864 59.9450496)


In [1]:
# result.explore()